Group Name: Suicide Squad

Calling the packages we will use for the project, along with the plotly username and api key

In [12]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
import seaborn as sns
import json
import requests
import warnings
from requests.auth import HTTPBasicAuth

def ignore_warn(*args, **kwargs):
    pass

warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

username = 'moonkang' # Replace with YOUR USERNAME
api_key = 'nKkigC26m95bqBAK52af' # Replace with YOUR API KEY

auth = HTTPBasicAuth(username, api_key)
headers = {'Plotly-Client-Platform': 'python'}

plotly.tools.set_credentials_file(username=username, api_key=api_key)

pd.set_option('display.max_columns', None)

This area is for deleting plots in plotly (run this block if the # of plots is already maxed out)

In [2]:
#for deleting plots in plotly
def get_pages(username, page_size):
    url = 'https://api.plot.ly/v2/folders/all?user='+username+'&page_size='+str(page_size)
    response = requests.get(url, auth=auth, headers=headers)
    if response.status_code != 200:
        return
    page = json.loads(response.content)
    yield page
    while True:
        resource = page['children']['next'] 
        if not resource:
            break
        response = requests.get(resource, auth=auth, headers=headers)
        if response.status_code != 200:
            break
        page = json.loads(response.content)
        yield page
        
def permanently_delete_files(username, page_size=500, filetype_to_delete='plot'):
    for page in get_pages(username, page_size):
        for x in range(0, len(page['children']['results'])):
            fid = page['children']['results'][x]['fid']
            res = requests.get('https://api.plot.ly/v2/files/' + fid, auth=auth, headers=headers)
            res.raise_for_status()
            if res.status_code == 200:
                json_res = json.loads(res.content)
                if json_res['filetype'] == filetype_to_delete:
                    # move to trash
                    requests.post('https://api.plot.ly/v2/files/'+fid+'/trash', auth=auth, headers=headers)
                    # permanently delete
                    requests.delete('https://api.plot.ly/v2/files/'+fid+'/permanent_delete', auth=auth, headers=headers)

permanently_delete_files(username, filetype_to_delete='plot')
permanently_delete_files(username, filetype_to_delete='grid')

Load the data from .csv and looking at the head of the data and dimensions

In [4]:
## load training data
train = pd.read_csv("train.csv", header = 0, index_col=None)
## load test data
test = pd.read_csv("test.csv", header = 0, index_col=None)

print(train.shape)
print(test.shape)
train.head()

(1460, 81)
(1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


Based on [Serigne's](https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard) kernel, we are dropping the ID column.

In [6]:
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

Added the sum of Basement and Livable Area as a column for the scatterplot matrix

In [7]:
train['LivBsmtArea']=train['TotalBsmtSF'] + train['GrLivArea']
test['LivBsmtArea']=test['TotalBsmtSF'] + test['GrLivArea']

Building the Scatterplot Matrix, just for us to see the relationship of the target variable with variables we think will have a big impact to the price

In [8]:
dataframe = train[['SalePrice', 'GrLivArea', 'TotalBsmtSF', 'LivBsmtArea']].copy()

fig = ff.create_scatterplotmatrix(dataframe, height=1000, width=1000, diag='histogram', size=3, title="House Train Variables")
py.iplot(fig)

Remove the outliers and log transform the sale price

In [9]:
#Outlier removal
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

#Log transform
train["LogSalePrice"] = np.log1p(train["SalePrice"])

Plot the data without the sales outliers

In [10]:
dataframe = train[['LogSalePrice', 'GrLivArea', 'TotalBsmtSF', 'LivBsmtArea']].copy()

fig = ff.create_scatterplotmatrix(dataframe, height=1000, width=1000, diag='histogram', size=3, title="House Train Variables")
py.iplot(fig)

In [13]:
SalePrice = go.Histogram(x=train['SalePrice'], cumulative=dict(enabled=True))
LogSalePrice = go.Histogram(x=train['LogSalePrice'], cumulative=dict(enabled=True))
fig = tools.make_subplots(rows=1, cols=2)

fig.append_trace(SalePrice, 1, 1)
fig.append_trace(LogSalePrice, 1, 2)

fig['layout'].update(height=400, width=800)

py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [48]:
corrmat = train.corr()
y = list(corrmat.columns.values)
x = list(corrmat.index.values)
z = [list(corrmat[x]) for x in corrmat]

In [60]:
trace = go.Heatmap(z=z, x=x, y=y, colorscale='Viridis')
data=[trace]
py.iplot(data)

# Cleaning process

Counting the missingness on each column

In [99]:
train_na = (train.isnull().sum() / len(train)) * 100
train_na = train_na.drop(train_na[train_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :train_na})
missing_data.head(10)

data = [go.Bar(x=train_na.index, y=train_na, marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5
        )))]

layout = go.Layout(
    xaxis=dict(tickangle=-90), title="% of Missingness"
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

In [25]:

#Handling features guided by data description

train.loc[:, "Alley"] = train.loc[:, "Alley"].fillna("None")
train.loc[:, "BsmtQual"] = train.loc[:, "BsmtQual"].fillna("NoBasmt")
train.loc[:, "BsmtCond"] = train.loc[:, "BsmtCond"].fillna("NoBasmt")
train.loc[:, "BsmtExposure"] = train.loc[:, "BsmtExposure"].fillna("NoBasmt")
train.loc[:, "BsmtFinType1"] = train.loc[:, "BsmtFinType1"].fillna("NoBasmt")
train.loc[:, "BsmtFinType2"] = train.loc[:, "BsmtFinType2"].fillna("NoBasmt")
train.loc[:, "BsmtFullBath"] = train.loc[:, "BsmtFullBath"].fillna(0)
train.loc[:, "BsmtHalfBath"] = train.loc[:, "BsmtHalfBath"].fillna(0)
train.loc[:, "BsmtUnfSF"] = train.loc[:, "BsmtUnfSF"].fillna(0)
train.loc[:, "Fence"] = train.loc[:, "Fence"].fillna("NoFnc")
train.loc[:, "FireplaceQu"] = train.loc[:, "FireplaceQu"].fillna("NoFrplc")
train.loc[:, "Fireplaces"] = train.loc[:, "Fireplaces"].fillna(0)
train.loc[:, "Functional"] = train.loc[:, "Functional"].fillna("Typ")
train.loc[:, "GarageType"] = train.loc[:, "GarageType"].fillna("NoGrg")
train.loc[:, "GarageFinish"] = train.loc[:, "GarageFinish"].fillna("NoGrg")
train.loc[:, "GarageQual"] = train.loc[:, "GarageQual"].fillna("NoGrg")
train.loc[:, "GarageCond"] = train.loc[:, "GarageCond"].fillna("NoGrg")
train.loc[:, "GarageArea"] = train.loc[:, "GarageArea"].fillna(0)
train.loc[:, "GarageCars"] = train.loc[:, "GarageCars"].fillna(0)
train.loc[:, "MiscFeature"] = train.loc[:, "MiscFeature"].fillna("NoMscFtr")
train.loc[:, "MiscVal"] = train.loc[:, "MiscVal"].fillna(0)
train.loc[:, "PoolQC"] = train.loc[:, "PoolQC"].fillna("NoPool")
train.loc[:, "PoolArea"] = train.loc[:, "PoolArea"].fillna(0)


In [70]:
#check ot frontage check vs lot area
tempdf = train[["LotFrontage", "LotArea"]]
tempdf.head()

trace = go.Scatter(
    x = train['LotFrontage'],
    y = train['LotArea'],
    mode = 'markers'
)

layout = go.Layout(
    xaxis=dict(
        title='Lot Frontage'),
    yaxis=dict(
        title='Lot Area')
)

data = [trace]

fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

In [80]:
index = train['LotFrontage'].index[train['LotFrontage'].apply(np.isnan)]
LFna_df = train.loc[index, :]
LFna_df.head(10)

#conclusion: the NaN in LotFrontage are zero because the column does not differentiate zero form NaN
#the column does not have zero as minimum based on describe
#NaN in lotfrontage = zero (fill with 0)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LivBsmtArea,LogSalePrice
7,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000,3197,12.206078
12,20,RL,NaN,12968,Pave,NaN,IR2,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,6,1962,1962,Hip,CompShg,HdBoard,Plywood,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,737,Unf,0,175,912,GasA,TA,Y,SBrkr,912,0,0,912,1,0,1,0,2,1,TA,4,Typ,0,NaN,Detchd,1962.0,Unf,1,352,TA,TA,Y,140,0,0,0,176,0,NaN,NaN,NaN,0,9,2008,WD,Normal,144000,1824,11.877576
14,20,RL,NaN,10920,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,MetalSd,MetalSd,BrkFace,212.0,TA,TA,CBlock,TA,TA,No,BLQ,733,Unf,0,520,1253,GasA,TA,Y,SBrkr,1253,0,0,1253,1,0,1,1,2,1,TA,5,Typ,1,Fa,Attchd,1960.0,RFn,1,352,TA,TA,Y,0,213,176,0,0,0,NaN,GdWo,NaN,0,5,2008,WD,Normal,157000,2506,11.964007
16,20,RL,NaN,11241,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,7,1970,1970,Gable,CompShg,Wd Sdng,Wd Sdng,BrkFace,180.0,TA,TA,CBlock,TA,TA,No,ALQ,578,Unf,0,426,1004,GasA,Ex,Y,SBrkr,1004,0,0,1004,1,0,1,0,2,1,TA,5,Typ,1,TA,Attchd,1970.0,Fin,2,480,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,Shed,700,3,2010,WD,Normal,149000,2008,11.911708
24,20,RL,NaN,8246,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,8,1968,2001,Gable,CompShg,Plywood,Plywood,None,0.0,TA,Gd,CBlock,TA,TA,Mn,Rec,188,ALQ,668,204,1060,GasA,Ex,Y,SBrkr,1060,0,0,1060,1,0,1,0,3,1,Gd,6,Typ,1,TA,Attchd,1968.0,Unf,1,270,TA,TA,Y,406,90,0,0,0,0,NaN,MnPrv,NaN,0,5,2010,WD,Normal,154000,2120,11.944714
31,20,RL,NaN,8544,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,6,1966,2006,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0,Unf,0,1228,1228,GasA,Gd,Y,SBrkr,1228,0,0,1228,0,0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1966.0,Unf,1,271,TA,TA,Y,0,65,0,0,0,0,NaN,MnPrv,NaN,0,6,2008,WD,Normal,149350,2456,11.914055
42,85,RL,NaN,9180,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,SFoyer,5,7,1983,1983,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,Gd,TA,Av,ALQ,747,LwQ,93,0,840,GasA,Gd,Y,SBrkr,884,0,0,884,1,0,1,0,2,1,Gd,5,Typ,0,NaN,Attchd,1983.0,RFn,2,504,TA,Gd,Y,240,0,0,0,0,0,NaN,MnPrv,NaN,0,12,2007,WD,Normal,144000,1724,11.877576
43,20,RL,NaN,9200,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,CollgCr,Norm,Norm,1Fam,1Story,5,6,1975,1980,Hip,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,Gd,TA,Av,LwQ,280,BLQ,491,167,938,GasA,TA,Y,SBrkr,938,0,0,938,1,0,1,0,3,1,TA,5,Typ,0,NaN,Detchd,1977.0,Unf,1,308,TA,TA,Y,145,0,0,0,0,0,NaN,MnPrv,NaN,0,7,2008,WD,Normal,130250,1876,11.777219
50,60,RL,NaN,13869,Pave,NaN,IR2,Lvl,AllPub,Corner,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1997,1997,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,Av,GLQ,182,Unf,0,612,794,GasA,Gd,Y,SBrkr,794,676,0,1470,0,1,2,0,3,1,TA,6,Typ,0,NaN,Attchd,1997.0,Fin,2,388,TA,TA,Y,0,75,0,0,0,0,NaN,NaN,NaN,0,7,2007,WD,Normal,177000,2264,12.083911
64,60,RL,NaN,9375,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Nor

In [26]:
# By missing at random, decided to fill NA with 0 for BedroomAbvGr as missingness can be interpreted to be directly 
# related to lack of vaue
train.loc[:, "BedroomAbvGr"] = train.loc[:, "BedroomAbvGr"].fillna(0)

# By missing at random, decided to fill NA with N for CentralAir as missingness can be interpreted to be directly 
# related to lack of Central Air
train.loc[:, "CentralAir"] = train.loc[:, "CentralAir"].fillna("N")

# By missing at random, decided to fill NA with Norm for Condition1 & Condition2 as missingness can be interpreted to be directly 
# related to lack of proximity to any conditions
train.loc[:, "Condition1"] = train.loc[:, "Condition1"].fillna("Norm")
train.loc[:, "Condition2"] = train.loc[:, "Condition2"].fillna("Norm")

# By missing at random, decided to fill NA with 0 for EnclosedPorch as missingness can be interpreted to be directly 
# related to lack of vaue
train.loc[:, "EnclosedPorch"] = train.loc[:, "EnclosedPorch"].fillna(0)

# By missing completely at random, decided to fill NA with TA (averaging) for ExterCond & ExterQual
# since we assume it will have minumum impact
train.loc[:, "ExterCond"] = train.loc[:, "ExterCond"].fillna("TA")
train.loc[:, "ExterQual"] = train.loc[:, "ExterQual"].fillna("TA")

# By missing at random, decided to fill NA with 0 for HalfBath as missingness can be interpreted to be directly 
# related to lack of vaue
train.loc[:, "HalfBath"] = train.loc[:, "HalfBath"].fillna(0)

# By missing completely at random, decided to fill NA with TA (averaging) for HeatingQC 
# since we assume it will have minumum impact
train.loc[:, "HeatingQC"] = train.loc[:, "HeatingQC"].fillna("TA")

# By missing at random, decided to fill NA with 0 for KitchenAbvGr as missingness can be interpreted to be directly 
# related to lack of vaue
train.loc[:, "KitchenAbvGr"] = train.loc[:, "KitchenAbvGr"].fillna(0)

# By missing completely at random, decided to fill NA with TA (averaging) for KitchenQual 
# since we assume it will have minumum impact
train.loc[:, "KitchenQual"] = train.loc[:, "KitchenQual"].fillna("TA")

# *****LotFrontage : NA most likely means no lot frontage - missing at random*****
train.loc[:, "LotFrontage"] = train.loc[:, "LotFrontage"].fillna(0)

# By missing completely at random, decided to fill NA with Reg (averaging) for LotShape 
# since we assume it will have minumum impact
train.loc[:, "LotShape"] = train.loc[:, "LotShape"].fillna("Reg")

# By missing at random, decided to fill NA with None and 0 for MasVnrType & MasVnrArea as missingness can be interpreted to be directly 
# related to lack of vaue
train.loc[:, "MasVnrType"] = train.loc[:, "MasVnrType"].fillna("None")
train.loc[:, "MasVnrArea"] = train.loc[:, "MasVnrArea"].fillna(0)

# By missing at random, decided to fill NA with 0 for OpenPorchSF as missingness can be interpreted to be directly 
# related to lack of vaue
train.loc[:, "OpenPorchSF"] = train.loc[:, "OpenPorchSF"].fillna(0)

# By missing at random, decided to fill NA with N for PavedDrive as missingness can be interpreted to be directly 
# related to No Paved Drive
train.loc[:, "PavedDrive"] = train.loc[:, "PavedDrive"].fillna("N")

# By missing completely at random, decided to fill NA with Normal (averaging) for SaleCondition 
# since we assume it will have minumum impact
train.loc[:, "SaleCondition"] = train.loc[:, "SaleCondition"].fillna("Normal")

# By missing at random, decided to fill NA with 0 for ScreenPorch as missingness can be interpreted to be directly 
# related to lack of vaue
train.loc[:, "ScreenPorch"] = train.loc[:, "ScreenPorch"].fillna(0)

# By missing at random, decided to fill NA with 0 for TotRmsAbvGrd as missingness can be interpreted to be directly 
# related to lack of vaue
train.loc[:, "TotRmsAbvGrd"] = train.loc[:, "TotRmsAbvGrd"].fillna(0)

# By missing at random, decided to fill NA with AllPub for Utilities as missingness can be interpreted to be directly 
# related to All public Utilities 
train.loc[:, "Utilities"] = train.loc[:, "Utilities"].fillna("AllPub")

# By missing at random, decided to fill NA with 0 for WoodDeckSF as missingness can be interpreted to be directly 
# related to lack of vaue
train.loc[:, "WoodDeckSF"] = train.loc[:, "WoodDeckSF"].fillna(0)

In [27]:
# There are variables with numerical vaues that after investigating in the Description we find that are actually
# best described as categories
# Changing the month sold and Year sold variables into categorical is a great example of 
# using dummy variables to adjust for seasonality

train = train.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                                       50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                                       80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                                       150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                       "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                   7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
                      })

train['YrSold'] = [str(x) for x in train['YrSold']]

In [28]:
# Encoding categorical features as ordered numbers after gathering inside of order from description

train = train.replace({"Alley" : {"None": 0, "Grvl" : 1, "Pave" : 2},
                       "BsmtCond" : {"NoBasmt" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "BsmtExposure" : {"NoBasmt" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
                       "BsmtFinType1" : {"NoBasmt" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtFinType2" : {"NoBasmt" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtQual" : {"NoBasmt" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
                       "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "FireplaceQu" : {"NoFrplc" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8},
                       "GarageCond" : {"NoGrg" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "GarageQual" : {"NoGrg" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                       "LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4},
                       "PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},
                       "PoolQC" : {"NoPool" : 0, "Fa" : 1, "TA" : 2, "Gd" : 3, "Ex" : 4},
                       "Street" : {"Grvl" : 1, "Pave" : 2},
                       "Utilities" : {"ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, "AllPub" : 4}}
                     )

In [ ]:
#THIS IS TO BE DISCUSSED LATER, LEAVE COMMENTED OUT




# # Create new features
# # 1* Simplifications of existing features
# train["SimplOverallQual"] = train.OverallQual.replace({1 : 1, 2 : 1, 3 : 1, # bad
#                                                        4 : 2, 5 : 2, 6 : 2, # average
#                                                        7 : 3, 8 : 3, 9 : 3, 10 : 3 # good
#                                                       })
# train["SimplOverallCond"] = train.OverallCond.replace({1 : 1, 2 : 1, 3 : 1, # bad
#                                                        4 : 2, 5 : 2, 6 : 2, # average
#                                                        7 : 3, 8 : 3, 9 : 3, 10 : 3 # good
#                                                       })
# train["SimplPoolQC"] = train.PoolQC.replace({1 : 1, 2 : 1, # average
#                                              3 : 2, 4 : 2 # good
#                                             })
# train["SimplGarageCond"] = train.GarageCond.replace({1 : 1, # bad
#                                                      2 : 1, 3 : 1, # average
#                                                      4 : 2, 5 : 2 # good
#                                                     })
# train["SimplGarageQual"] = train.GarageQual.replace({1 : 1, # bad
#                                                      2 : 1, 3 : 1, # average
#                                                      4 : 2, 5 : 2 # good
#                                                     })
# train["SimplFireplaceQu"] = train.FireplaceQu.replace({1 : 1, # bad
#                                                        2 : 1, 3 : 1, # average
#                                                        4 : 2, 5 : 2 # good
#                                                       })
# train["SimplFireplaceQu"] = train.FireplaceQu.replace({1 : 1, # bad
#                                                        2 : 1, 3 : 1, # average
#                                                        4 : 2, 5 : 2 # good
#                                                       })
# train["SimplFunctional"] = train.Functional.replace({1 : 1, 2 : 1, # bad
#                                                      3 : 2, 4 : 2, # major
#                                                      5 : 3, 6 : 3, 7 : 3, # minor
#                                                      8 : 4 # typical
#                                                     })
# train["SimplKitchenQual"] = train.KitchenQual.replace({1 : 1, # bad
#                                                        2 : 1, 3 : 1, # average
#                                                        4 : 2, 5 : 2 # good
#                                                       })
# train["SimplHeatingQC"] = train.HeatingQC.replace({1 : 1, # bad
#                                                    2 : 1, 3 : 1, # average
#                                                    4 : 2, 5 : 2 # good
#                                                   })
# train["SimplBsmtFinType1"] = train.BsmtFinType1.replace({1 : 1, # unfinished
#                                                          2 : 1, 3 : 1, # rec room
#                                                          4 : 2, 5 : 2, 6 : 2 # living quarters
#                                                         })
# train["SimplBsmtFinType2"] = train.BsmtFinType2.replace({1 : 1, # unfinished
#                                                          2 : 1, 3 : 1, # rec room
#                                                          4 : 2, 5 : 2, 6 : 2 # living quarters
#                                                         })
# train["SimplBsmtCond"] = train.BsmtCond.replace({1 : 1, # bad
#                                                  2 : 1, 3 : 1, # average
#                                                  4 : 2, 5 : 2 # good
#                                                 })
# train["SimplBsmtQual"] = train.BsmtQual.replace({1 : 1, # bad
#                                                  2 : 1, 3 : 1, # average
#                                                  4 : 2, 5 : 2 # good
#                                                 })
# train["SimplExterCond"] = train.ExterCond.replace({1 : 1, # bad
#                                                    2 : 1, 3 : 1, # average
#                                                    4 : 2, 5 : 2 # good
#                                                   })
# train["SimplExterQual"] = train.ExterQual.replace({1 : 1, # bad
#                                                    2 : 1, 3 : 1, # average
#                                                    4 : 2, 5 : 2 # good
#                                                   })

# # 2* Combinations of existing features
# # Overall quality of the house
# train["OverallGrade"] = train["OverallQual"] * train["OverallCond"]
# # Overall quality of the garage
# train["GarageGrade"] = train["GarageQual"] * train["GarageCond"]
# # Overall quality of the exterior
# train["ExterGrade"] = train["ExterQual"] * train["ExterCond"]
# # Overall kitchen score
# train["KitchenScore"] = train["KitchenAbvGr"] * train["KitchenQual"]
# # Overall fireplace score
# train["FireplaceScore"] = train["Fireplaces"] * train["FireplaceQu"]
# # Overall garage score
# train["GarageScore"] = train["GarageArea"] * train["GarageQual"]
# # Overall pool score
# train["PoolScore"] = train["PoolArea"] * train["PoolQC"]
# # Simplified overall quality of the house
# train["SimplOverallGrade"] = train["SimplOverallQual"] * train["SimplOverallCond"]
# # Simplified overall quality of the exterior
# train["SimplExterGrade"] = train["SimplExterQual"] * train["SimplExterCond"]
# # Simplified overall pool score
# train["SimplPoolScore"] = train["PoolArea"] * train["SimplPoolQC"]
# # Simplified overall garage score
# train["SimplGarageScore"] = train["GarageArea"] * train["SimplGarageQual"]
# # Simplified overall fireplace score
# train["SimplFireplaceScore"] = train["Fireplaces"] * train["SimplFireplaceQu"]
# # Simplified overall kitchen score
# train["SimplKitchenScore"] = train["KitchenAbvGr"] * train["SimplKitchenQual"]
# # Total number of bathrooms
# train["TotalBath"] = train["BsmtFullBath"] + (0.5 * train["BsmtHalfBath"]) + \
# train["FullBath"] + (0.5 * train["HalfBath"])




##### DO NOT Uncomment this. I already made this column in the scatterplot above #####
# # Total SF for house (incl. basement)
# train["AllSF"] = train["GrLivArea"] + train["TotalBsmtSF"]


# # Total SF for 1st + 2nd floors
# train["AllFlrsSF"] = train["1stFlrSF"] + train["2ndFlrSF"]
# # Total SF for porch
# train["AllPorchSF"] = train["OpenPorchSF"] + train["EnclosedPorch"] + \
# train["3SsnPorch"] + train["ScreenPorch"]
# # Has masonry veneer or not
# train["HasMasVnr"] = train.MasVnrType.replace({"BrkCmn" : 1, "BrkFace" : 1, "CBlock" : 1, 
#                                                "Stone" : 1, "None" : 0})
# # House completed before sale or not
# train["BoughtOffPlan"] = train.SaleCondition.replace({"Abnorml" : 0, "Alloca" : 0, "AdjLand" : 0, 
#                                                       "Family" : 0, "Normal" : 0, "Partial" : 1})